In [1]:
import os

from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

load_dotenv()

# llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOpenAI(model=os.getenv("DEFAULT_MODEL"))

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='저는 잘 지내고 있어요. 어떻게 지내고 계세요? 다른 도움이 필요하신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 15, 'total_tokens': 36, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.196730825, 'prompt_time': 0.002591229, 'completion_time': 0.048820683, 'total_time': 0.051411912}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_5d3e4e58e1', 'id': 'chatcmpl-bf9d80b8-afa4-45c8-8c7d-f50e32e36d54', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--70bcd583-fe05-4a2d-8344-af47eb62bb15-0', usage_metadata={'input_tokens': 15, 'output_tokens': 21, 'total_tokens': 36, 'input_token_details': {}, 'output_token_details': {}})

In [2]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time


In [4]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [5]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'kfebb2h50', 'function': {'arguments': '{"location":"Busan","timezone":"Asia/Seoul"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 758, 'total_tokens': 792, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.044782806, 'prompt_time': 0.022565055, 'completion_time': 0.077682286, 'total_time': 0.100247341}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_88e13d3851', 'id': 'chatcmpl-ff742999-d42b-4f13-be20-4290e4cc93a8', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e5ea359f-8c8b-443f-bfa5-e8b52593c008-0', tool_calls=[{'name': 'ge

In [6]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'location': 'Busan', 'timezone': 'Asia/Seoul'}
Asia/Seoul (Busan) 현재시각 2025-08-16 23:04:47 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'kfebb2h50', 'function': {'arguments': '{"location":"Busan","timezone":"Asia/Seoul"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 758, 'total_tokens': 792, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.044782806, 'prompt_time': 0.022565055, 'completion_time': 0.077682286, 'total_time': 0.100247341}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_88e13d3851', 'id': 'chatcmpl-ff742999-d42b-4f13-be20-4290e4cc93a8', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e5ea359f-8c8b-443f-bfa5-e8b52593c008-0', tool_calls=[{'name': '

In [7]:
llm_with_tools.invoke(messages)

AIMessage(content='부산은 현재 23시 4분입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 832, 'total_tokens': 844, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.047616453, 'prompt_time': 0.025698387, 'completion_time': 0.028139278, 'total_time': 0.053837665}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_5d3e4e58e1', 'id': 'chatcmpl-aa799a32-e499-4865-9c92-e13a30d9b625', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a9bf941b-52a8-44ef-a046-193e01868bb4-0', usage_metadata={'input_tokens': 832, 'output_tokens': 12, 'total_tokens': 844, 'input_token_details': {}, 'output_token_details': {}})